In [2]:
import sys
#sys.path.insert(1, "C:\pdfminer-20140328")


In [8]:
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.converter import TextConverter
from pdfminer.layout import LAParams
from pdfminer.pdfpage import PDFPage
from cStringIO import StringIO

def convert_pdf_to_txt(path):
    rsrcmgr = PDFResourceManager()
    retstr = StringIO()
    codec = 'utf-8'
    laparams = LAParams()
    device = TextConverter(rsrcmgr, retstr, codec=codec, laparams=laparams)
    fp = file(path, 'rb')
    interpreter = PDFPageInterpreter(rsrcmgr, device)
    password = ""
    maxpages = 0
    caching = True
    pagenos=set()

    for page in PDFPage.get_pages(fp, pagenos, maxpages=maxpages, password=password,caching=caching, check_extractable=True):
        interpreter.process_page(page)

    text = retstr.getvalue()

    fp.close()
    device.close()
    retstr.close()
    return text



In [9]:
from os import path
from glob import glob 

def find_ext(dr, ext):
    return glob(path.join(dr,"*.{}".format(ext)))



In [12]:
liste = find_ext("./tazadata/","pdf")
for e in liste:
    text_file = open(str(e) + ".txt", "w")
    text_file.write(convert_pdf_to_txt(e))
    text_file.close()

In [129]:
# -*- coding: utf-8 -*-
import pandas as pd

liste = find_ext("./tazadata/","txt")

valuesTaza = []

for f in liste:
    lines = [line.rstrip('\n') for line in open(f)]
    #print str(f)[-29:]
    
    idxMalla = lines.index("Malla")
    idxItem = lines.index("Item")
    idxDefectos = lines.index("Defectos")
    idxCarac = lines.index("Características")
    idxTaza = lines.index("Tabla 5. Descripción de la taza")
    chapterIndexes = [idxMalla,idxItem,idxDefectos,idxCarac,idxTaza]
    
    #################################################
    
    # Granulomémtrie
    
    Tabla1Item = []
    Tabla1val= []
    
    for i in range(0,8):
        idxM = idxMalla + 2 + i
        idxVal = idxMalla + 12 + i
        
        colname = "Malla " + lines[idxM]
        Tabla1Item.append(colname)
        Tabla1val.append(lines[idxVal])
        #print("Malla: {}  value: {}".format(lines[idxM], lines[idxVal]))
    
    #print len(Tabla1Item)
    #print len(Tabla1val)
    
    #################################################
    
    # Caractéristiques physiques - éspaces vides aléatoires
    Tabla3Item = []
    Tabla3val = []
    
    for i in range(0,7):
        idxIt = idxItem + 4 + i
        Tabla3Item.append(lines[idxIt])
        #print("Item: {}".format(lines[idxIt]))
    
    idxItemFactor = lines.index("Factor de Rendimiento")
    
    s = ""
    for i in range(idxItemFactor,idxDefectos -1):
        # + 1 pour éviter le "factor de rendimiento"
        if str(lines[i+1]) == s:
            continue
        
        Tabla3val.append(lines[i+1])
        #print("Value: {}".format(lines[i+1]))
    
    # supprime le 1er (si c'est 8, le titre de la collone se met en première place)
    if len(Tabla3val) == 8:
        Tabla3val = Tabla3val[1:]
    
    #print len(Tabla3Item)
    #print len(Tabla3val)
    
    
    
    #################################################
    
    # Defectos
    
    Tabla2Item = []
    Tabla2val = []
    
    # index of the second (%) -> because right before the defectos values
    indices = [i for i, x in enumerate(lines) if x == "(%)"]
    indice = indices[1]
    
    for i in range(0, 20):
        idxIt = idxDefectos + 2 + i
        idxVal = indice + 1 + i
        
        Tabla2Item.append(lines[idxIt])
        Tabla2val.append(lines[idxVal])
        
    Tabla2val = ' '.join(Tabla2val).split()
    Tabla2Item = Tabla2Item[:-1]
    
    
    
    #print len(Tabla2Item)
    #print len(Tabla2val)
    
    #################################################
    
    # Caracteristicas
    
    Tabla4Item = []
    Tabla4val = []
    
    for i in range(0,10):
        idxIt = idxCarac + 4 + i
        idxVal = idxCarac + 17 + i
        
        Tabla4Item.append(lines[idxIt])
        Tabla4val.append(lines[idxVal])
    #print len(Tabla4Item)
    #print len(Tabla4val)
    
    #################################################
    
    # Description de la taza
    
    possibleValues = ["Limpia","Agrio","Vinagre","Fermento","Stinker","Metálico",
                      "Maderoso","Acre","Reposo","Terroso","Moho","Fenol",
                      "Químico","Humo","Contaminado"]
    
    Tabla5Item = ["Taza 1","Taza 2","Taza 3","Taza 4","Taza 5"]
    Tabla5val = []
    
    found = False
    while found == False:
        idxTaza = idxTaza + 1
        if str(lines[idxTaza]) in possibleValues:
            found = True
    
    
    for i in range(0,5):
        idxVal = idxTaza  + i
        Tabla5val.append(lines[idxVal])

        
    items = Tabla1Item + Tabla2Item + Tabla3Item + Tabla4Item + Tabla5Item
    values = Tabla1val + Tabla2val + Tabla3val + Tabla4val + Tabla5val
    
    valuesTaza.append(values)
    

df = pd.DataFrame.from_records(valuesTaza, columns=items)
print(df)

   Malla 18 Malla 17 Malla 16 Malla 15 Malla 14 Malla 13 Malla 12 Malla 0  \
0     35.52    29.41    17.94    11.12     4.81     1.05      0.1    0.05   
1     40.05    28.93    17.97     8.29     4.07     0.45      0.1    0.15   
2     37.62    29.64    19.18     8.04     4.54     0.84      0.1    0.05   
3     42.68    27.79    17.06     7.02     4.15     1.09     0.15    0.05   
4     44.92    26.52     17.3     6.13     4.24      0.7     0.15    0.05   
5      39.5    30.67    16.37     8.92      3.3     1.04     0.15    0.05   
6     31.07    29.38    18.65    11.82     6.43     2.34      0.2     0.1   
7      51.4    27.17    14.01     5.18     1.62     0.48      0.1    0.05   
8     46.29    27.22    16.07     7.08     2.56     0.54      0.1    0.15   
9     37.37    31.29    19.97     6.79     3.17     1.21     0.15    0.05   
10     45.2    26.78     15.3     8.12     3.32     0.94      0.2    0.15   
11    43.84    26.77    16.18     8.71     2.52     1.48      0.3     0.2   

In [127]:
with pd.option_context('display.max_rows', None, 'display.max_columns', 49):
    print(df)

73
       18     17     16     15    14    13    12     0 Negros y vinagres  \
0   35.52  29.41  17.94  11.12  4.81  1.05   0.1  0.05               0.3   
1   40.05  28.93  17.97   8.29  4.07  0.45   0.1  0.15               0.3   
2   37.62  29.64  19.18   8.04  4.54  0.84   0.1  0.05              0.39   
3   42.68  27.79  17.06   7.02  4.15  1.09  0.15  0.05              0.15   
4   44.92  26.52   17.3   6.13  4.24   0.7  0.15  0.05               0.0   
5    39.5  30.67  16.37   8.92   3.3  1.04  0.15  0.05              0.25   
6   31.07  29.38  18.65  11.82  6.43  2.34   0.2   0.1               0.2   
7    51.4  27.17  14.01   5.18  1.62  0.48   0.1  0.05              0.33   
8   46.29  27.22  16.07   7.08  2.56  0.54   0.1  0.15              0.79   
9   37.37  31.29  19.97   6.79  3.17  1.21  0.15  0.05              1.11   
10   45.2  26.78   15.3   8.12  3.32  0.94   0.2  0.15               0.3   
11  43.84  26.77  16.18   8.71  2.52  1.48   0.3   0.2              0.89   
12  60.89